# Portfolio Asset Clustering Project - FIXED VERSION

This notebook consolidates the working approach with proper preprocessing, PCA optimization, and clustering.

## 1. Import Libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from scipy.stats.mstats import winsorize

# Clustering Algorithms
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering, MiniBatchKMeans, Birch
from sklearn.mixture import GaussianMixture

# Metrics and Validation
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy import stats

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Load and Prepare Data

In [18]:
# Load data
df = pd.read_excel('NY165_stock_data.xlsx', sheet_name='Stock_Data')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Ticker', 'Date'])

print(f"Unique stocks: {df['Ticker'].nunique()}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"Trading days: {df['Date'].nunique()}")

# Store metadata for later use
meta_data = pd.DataFrame()
if 'sector' in df.columns:
    meta_data = df.groupby('Ticker')[['sector', 'industry', 'company']].first()
    print(f"\nSectors: {df['sector'].nunique()}")
    print(df['sector'].value_counts())

Unique stocks: 165
Date range: 2023-12-01 to 2025-11-28
Trading days: 500


## 3. Feature Calculation

In [19]:
def calculate_advanced_features(group):
    """Calculate comprehensive features for clustering with consistent output structure."""
    
    features = {}
    returns = group['Daily_Return'].dropna()
    prices = group['Close']
    
    # Check for sufficient data (at least 6 months)
    if len(returns) < 126:
        return pd.Series(features)
    
    # --- RETURNS ---
    features['return_mean'] = returns.mean() * 252
    features['return_6m'] = (prices.iloc[-1] / prices.iloc[-min(126, len(prices))] - 1) if len(prices) >= 126 else 0
    
    # --- VOLATILITY ---
    features['volatility'] = returns.std() * np.sqrt(252)
    features['downside_vol'] = returns[returns < 0].std() * np.sqrt(252) if (returns < 0).any() else 0
    
    vol_rolling = returns.rolling(20).std() * np.sqrt(252)
    features['vol_stability'] = vol_rolling.std() / vol_rolling.mean() if vol_rolling.mean() > 0 else 0
    
    # --- RISK ADJUSTED ---
    # Add small epsilon to avoid division by zero
    features['sharpe'] = features['return_mean'] / (features['volatility'] + 1e-6)
    features['sortino'] = features['return_mean'] / (features['downside_vol'] + 1e-6)
    
    # --- TAIL RISK & DIST ---
    features['var_95'] = abs(returns.quantile(0.05))
    features['skewness'] = returns.skew()
    
    # --- LIQUIDITY ---
    if 'Volume' in group.columns:
        volume = group['Volume']
        # Use raw volume here; we log transform later in preprocessing
        features['avg_volume'] = volume.mean() 
        features['volume_stability'] = volume.std() / (volume.mean() + 1e-6)
    
    return pd.Series(features)

print("Calculating features for each stock...")
stock_features = df.groupby('Ticker').apply(calculate_advanced_features)

# Handle MultiIndex Output if necessary
if isinstance(stock_features, pd.Series) or isinstance(stock_features.index, pd.MultiIndex):
    stock_features = stock_features.unstack()

# Drop rows where all features are NaN (stocks with insufficient history)
stock_features = stock_features.dropna(how='all')

print(f"\n✓ Features calculated for {len(stock_features)} stocks")
print(f"✓ Total features: {stock_features.shape[1]}")
print(f"\nFeature list:")
for i, col in enumerate(stock_features.columns, 1):
    print(f"  {i}. {col}")

# Save raw features
stock_features.to_excel('01_raw_features.xlsx', index=True)
print("\n✓ Saved raw features to 01_raw_features.xlsx")

Calculating features for each stock...

✓ Features calculated for 163 stocks
✓ Total features: 11

Feature list:
  1. avg_volume
  2. downside_vol
  3. return_6m
  4. return_mean
  5. sharpe
  6. skewness
  7. sortino
  8. var_95
  9. vol_stability
  10. volatility
  11. volume_stability

✓ Saved raw features to 01_raw_features.xlsx


## 4. Preprocessing & Optimization Pipeline

This is the KEY section that makes clustering work well!

In [20]:
def preprocess_and_optimize(df_features):
    """
    Cleans, selects, and transforms features to maximize silhouette score.
    This pipeline is CRITICAL for good clustering results!
    """
    print("="*80)
    print("PREPROCESSING PIPELINE")
    print("="*80)
    
    # 1. Drop NaN rows
    df_clean = df_features.dropna()
    print(f"\n[1] Initial stocks: {len(df_features)} → After dropping NaNs: {len(df_clean)}")
    
    if len(df_clean) < 30:
        print("\n⚠ WARNING: Too few stocks after cleaning. Consider relaxing filters.")
        return None
    
    # 2. Winsorization (Clip Outliers)
    # Extreme outliers squash clusters. We clip top/bottom 2.5%
    print("\n[2] Clipping outliers (Winsorization 2.5%)...")
    outliers_clipped = 0
    for col in df_clean.columns:
        q025 = df_clean[col].quantile(0.025)
        q975 = df_clean[col].quantile(0.975)
        before = ((df_clean[col] < q025) | (df_clean[col] > q975)).sum()
        df_clean[col] = winsorize(df_clean[col], limits=[0.025, 0.025])
        outliers_clipped += before
    print(f"    → Clipped {outliers_clipped} outlier values")
        
    # 3. Log Transform Skewed Features
    # Volatility and Volume are often log-normal. Transforming them helps KMeans.
    print("\n[3] Log-transforming skewed features...")
    skewed_cols = ['avg_volume', 'volatility', 'downside_vol', 'var_95']
    transformed = []
    for col in skewed_cols:
        if col in df_clean.columns:
            # Ensure positive before log
            if (df_clean[col] <= 0).any():
                df_clean[col] = np.log1p(df_clean[col] - df_clean[col].min() + 1e-6)
            else:
                df_clean[col] = np.log1p(df_clean[col])
            transformed.append(col)
    print(f"    → Transformed: {transformed}")
    
    # 4. Feature Selection: Remove Highly Correlated Features
    # High correlation adds weight to a specific dimension without adding info.
    print("\n[4] Removing highly correlated features (>0.85)...")
    corr_matrix = df_clean.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
    
    if to_drop:
        print(f"    → Dropping: {to_drop}")
        df_selected = df_clean.drop(columns=to_drop)
    else:
        print(f"    → No highly correlated features found")
        df_selected = df_clean
    
    # 5. Scaling
    # RobustScaler is better than StandardScaler for financial data (less sensitive to outliers)
    print("\n[5] Scaling features (RobustScaler)...")
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(df_selected)
    X_scaled = pd.DataFrame(X_scaled, columns=df_selected.columns, index=df_selected.index)
    
    print("\n" + "="*80)
    print("PREPROCESSING SUMMARY")
    print("="*80)
    print(f"Original features:        {df_features.shape[1]}")
    print(f"Outliers clipped:         {outliers_clipped}")
    print(f"Features after corr drop: {df_selected.shape[1]}")
    print(f"Stocks in analysis:       {len(X_scaled)}")
    print(f"\nFinal features: {list(X_scaled.columns)}")
    
    return X_scaled

# Run preprocessing
X_processed = preprocess_and_optimize(stock_features)

if X_processed is not None:
    X_processed.to_csv('02_preprocessed_features.csv')
    print("\n✓ Saved preprocessed features to 02_preprocessed_features.csv")

PREPROCESSING PIPELINE

[1] Initial stocks: 163 → After dropping NaNs: 163

[2] Clipping outliers (Winsorization 2.5%)...
    → Clipped 110 outlier values

[3] Log-transforming skewed features...
    → Transformed: ['avg_volume', 'volatility', 'downside_vol', 'var_95']

[4] Removing highly correlated features (>0.85)...
    → Dropping: ['sharpe', 'sortino', 'var_95', 'volatility']

[5] Scaling features (RobustScaler)...

PREPROCESSING SUMMARY
Original features:        11
Outliers clipped:         110
Features after corr drop: 7
Stocks in analysis:       163

Final features: ['avg_volume', 'downside_vol', 'return_6m', 'return_mean', 'skewness', 'vol_stability', 'volume_stability']

✓ Saved preprocessed features to 02_preprocessed_features.csv


## 5. PCA + K-Means Optimization (Grid Search)

This jointly optimizes both PCA components AND number of clusters!

In [21]:
def find_best_clusters(X_scaled):
    """
    Grid search for best K and PCA components.
    This is what makes the difference between 0.25 and 0.5 silhouette scores!
    """
    print("\n" + "="*80)
    print("OPTIMIZING CLUSTERS (PCA Components × K)")
    print("="*80)
    print("\nTesting different combinations...\n")
    
    best_score = -1
    best_k = 0
    best_pca = 0
    best_labels = None
    best_pca_data = None
    best_pca_model = None
    
    results_log = []
    
    # Try different PCA components (2 to min(6, n_features))
    n_features = X_scaled.shape[1]
    pca_options = list(range(2, min(7, n_features) + 1))
    
    for n_components in pca_options:
        pca = PCA(n_components=n_components, random_state=42)
        X_pca = pca.fit_transform(X_scaled)
        
        # Try different K (3 to 8)
        # Avoid K=2 as it often just splits market vs non-market
        for k in range(3, 9):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=20, max_iter=500)
            labels = kmeans.fit_predict(X_pca)
            
            # Calculate metrics
            sil_score = silhouette_score(X_pca, labels)
            cal_score = calinski_harabasz_score(X_pca, labels)
            db_score = davies_bouldin_score(X_pca, labels)
            
            # Balance score
            sizes = pd.Series(labels).value_counts().values
            balance = sizes.min() / sizes.max()
            
            results_log.append({
                'pca': n_components,
                'k': k,
                'silhouette': sil_score,
                'calinski': cal_score,
                'davies_bouldin': db_score,
                'balance': balance
            })
            
            # Update best if this is better
            # We prioritize silhouette but also consider balance
            if sil_score > best_score and balance > 0.3:  # Balance threshold
                best_score = sil_score
                best_k = k
                best_pca = n_components
                best_labels = labels
                best_pca_data = X_pca
                best_pca_model = pca
    
    # Display top 10 results
    results_df = pd.DataFrame(results_log).sort_values('silhouette', ascending=False)
    print("\nTop 10 Combinations:")
    print("="*90)
    print(f"{'PCA':<5} {'K':<4} {'Silhouette':<12} {'Balance':<10} {'Calinski-H':<12} {'Davies-B':<12}")
    print("="*90)
    for i, row in results_df.head(10).iterrows():
        marker = " ← SELECTED" if (row['pca'] == best_pca and row['k'] == best_k) else ""
        print(f"{row['pca']:<5.0f} {row['k']:<4.0f} {row['silhouette']:<12.4f} {row['balance']:<10.3f} "
              f"{row['calinski']:<12.2f} {row['davies_bouldin']:<12.4f}{marker}")
    
    print("\n" + "="*80)
    print("🏆 OPTIMAL CONFIGURATION FOUND")
    print("="*80)
    print(f"Silhouette Score:     {best_score:.4f}")
    print(f"Number of Clusters:   {best_k}")
    print(f"PCA Components:       {best_pca}")
    print(f"Variance Explained:   {best_pca_model.explained_variance_ratio_.sum()*100:.2f}%")
    
    # Cluster sizes
    sizes = pd.Series(best_labels).value_counts().sort_index()
    print(f"\nCluster Distribution:")
    for cluster, count in sizes.items():
        print(f"  Cluster {cluster}: {count:3d} stocks ({count/len(best_labels)*100:5.1f}%)")
    
    return best_labels, best_pca_data, best_score, best_k, best_pca_model

# Run optimization
if X_processed is not None:
    optimal_labels, X_pca_optimal, optimal_score, optimal_k, pca_model = find_best_clusters(X_processed)
    
    # Save results
    results_df = X_processed.copy()
    results_df['Cluster'] = optimal_labels
    
    # Add metadata if available
    if not meta_data.empty:
        results_df = results_df.join(meta_data)
    
    results_df.to_excel(f'03_optimized_clusters_k{optimal_k}_score{optimal_score:.3f}.xlsx')
    print(f"\n✓ Saved results to 03_optimized_clusters_k{optimal_k}_score{optimal_score:.3f}.xlsx")


OPTIMIZING CLUSTERS (PCA Components × K)

Testing different combinations...


Top 10 Combinations:
PCA   K    Silhouette   Balance    Calinski-H   Davies-B    
2     3    0.5845       0.042      239.84       0.7104      
3     3    0.5255       0.043      165.44       0.8995      
4     3    0.4964       0.043      132.11       1.0479      
5     3    0.4659       0.043      116.04       1.1035      
6     3    0.4509       0.043      106.96       1.1462      
2     8    0.4132       0.030      264.64       0.6460      
2     7    0.4037       0.029      250.69       0.6598      
2     6    0.4012       0.088      246.30       0.7260      
7     3    0.3957       0.044      101.65       1.1918      
2     4    0.3655       0.080      251.25       0.8029      

🏆 OPTIMAL CONFIGURATION FOUND
Silhouette Score:     -1.0000
Number of Clusters:   0
PCA Components:       0


AttributeError: 'NoneType' object has no attribute 'explained_variance_ratio_'

## 6. Visualization

In [ ]:
if X_processed is not None and optimal_labels is not None:
    # Create 2D visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: PCA space (first 2 components)
    colors = plt.cm.tab10(np.linspace(0, 1, optimal_k))
    
    for cluster in range(optimal_k):
        mask = optimal_labels == cluster
        axes[0].scatter(X_pca_optimal[mask, 0], X_pca_optimal[mask, 1],
                    c=[colors[cluster]], label=f'Cluster {cluster}',
                    s=80, alpha=0.6, edgecolors='black', linewidth=0.5)
        
        # Centroid
        centroid = X_pca_optimal[mask].mean(axis=0)
        axes[0].scatter(centroid[0], centroid[1], marker='X', s=400,
                    c=[colors[cluster]], edgecolors='black', linewidth=2, zorder=10)
    
    var_ratio = pca_model.explained_variance_ratio_
    axes[0].set_xlabel(f'PC1 ({var_ratio[0]*100:.1f}%)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel(f'PC2 ({var_ratio[1]*100:.1f}%)', fontsize=12, fontweight='bold')
    axes[0].set_title(f'Cluster Visualization - PCA Space\n'
                     f'Silhouette Score: {optimal_score:.3f}', fontsize=13, fontweight='bold')
    axes[0].legend(loc='best', fontsize=9)
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Feature importance (PCA loadings)
    loadings = pca_model.components_[:2].T
    feature_importance = np.abs(loadings).sum(axis=1)
    feature_names = X_processed.columns
    
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=True)
    
    axes[1].barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
    axes[1].set_xlabel('Cumulative Absolute Loading (PC1 + PC2)', fontsize=11, fontweight='bold')
    axes[1].set_title('Feature Importance in PCA', fontsize=13, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig(f'04_visualization_k{optimal_k}_score{optimal_score:.3f}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Visualization saved!")

## 7. Cluster Profiling

In [ ]:
if X_processed is not None and optimal_labels is not None:
    print("\n" + "="*80)
    print("CLUSTER PROFILING")
    print("="*80)
    
    # Add cluster labels to original features
    profile_df = stock_features.loc[X_processed.index].copy()
    profile_df['Cluster'] = optimal_labels
    
    for cluster in sorted(profile_df['Cluster'].unique()):
        cluster_data = profile_df[profile_df['Cluster'] == cluster]
        
        print(f"\n{'='*80}")
        print(f"CLUSTER {cluster} - {len(cluster_data)} stocks ({len(cluster_data)/len(profile_df)*100:.1f}%)")
        print(f"{'='*80}")
        
        # Calculate statistics
        numeric_cols = [col for col in profile_df.columns if col != 'Cluster']
        cluster_means = cluster_data[numeric_cols].mean()
        overall_means = profile_df[numeric_cols].mean()
        
        print("\nKey Characteristics (vs overall average):")
        print(f"{'Feature':<20} {'Cluster Avg':<15} {'Diff %':<12} {'Direction'}")
        print("-" * 80)
        
        for col in numeric_cols:
            val = cluster_means[col]
            overall = overall_means[col]
            diff = ((val / overall - 1) * 100) if overall != 0 else 0
            
            if abs(diff) > 10:  # Only show significant differences
                direction = '↑↑ High' if diff > 25 else '↑ Above' if diff > 10 else '↓ Below' if diff < -10 else '↓↓ Low' if diff < -25 else '≈ Average'
                print(f"{col:<20} {val:<15.4f} {diff:<12.1f} {direction}")
        
        # Show sample stocks if we have company names
        if not meta_data.empty and 'company' in meta_data.columns:
            sample_stocks = cluster_data.join(meta_data['company']).head(5)
            print(f"\nSample Stocks:")
            for ticker in sample_stocks.index:
                company = sample_stocks.loc[ticker, 'company'] if 'company' in sample_stocks.columns else ticker
                print(f"  • {ticker}: {company}")

## 8. Algorithm Comparison (Optional)

Now let's see if other algorithms can beat K-Means!

In [ ]:
if X_processed is not None and optimal_k is not None:
    print("\n" + "="*80)
    print("COMPARING CLUSTERING ALGORITHMS")
    print("="*80)
    print(f"\nUsing optimal k={optimal_k} from K-Means optimization\n")
    
    clustering_results = {}
    
    # Use PCA data for fair comparison
    X_for_comparison = X_pca_optimal
    
    # Define algorithms to test
    algorithms = {
        'K-Means (Optimized)': KMeans(n_clusters=optimal_k, random_state=42, n_init=20),
        'Mini-Batch K-Means': MiniBatchKMeans(n_clusters=optimal_k, random_state=42, n_init=20),
        'Hierarchical (Ward)': AgglomerativeClustering(n_clusters=optimal_k, linkage='ward'),
        'Hierarchical (Average)': AgglomerativeClustering(n_clusters=optimal_k, linkage='average'),
        'Gaussian Mixture': GaussianMixture(n_components=optimal_k, random_state=42, n_init=10),
        'Spectral': SpectralClustering(n_clusters=optimal_k, random_state=42, affinity='nearest_neighbors'),
    }
    
    for name, model in algorithms.items():
        try:
            labels = model.fit_predict(X_for_comparison)
            
            sil_score = silhouette_score(X_for_comparison, labels)
            cal_score = calinski_harabasz_score(X_for_comparison, labels)
            db_score = davies_bouldin_score(X_for_comparison, labels)
            
            sizes = pd.Series(labels).value_counts().values
            balance = sizes.min() / sizes.max()
            
            clustering_results[name] = {
                'labels': labels,
                'silhouette': sil_score,
                'calinski': cal_score,
                'davies_bouldin': db_score,
                'balance': balance
            }
            
        except Exception as e:
            print(f"⚠ {name} failed: {str(e)[:60]}")
            continue
    
    # Create comparison table
    comparison_df = pd.DataFrame({
        'Algorithm': list(clustering_results.keys()),
        'Silhouette': [v['silhouette'] for v in clustering_results.values()],
        'Balance': [v['balance'] for v in clustering_results.values()],
        'Calinski-H': [v['calinski'] for v in clustering_results.values()],
        'Davies-B': [v['davies_bouldin'] for v in clustering_results.values()]
    }).sort_values('Silhouette', ascending=False)
    
    print("\nALGORITHM COMPARISON RESULTS")
    print("="*100)
    print(f"{'Algorithm':<25} {'Silhouette':<15} {'Balance':<12} {'Calinski-H':<15} {'Davies-B':<12}")
    print("="*100)
    
    for idx, row in comparison_df.iterrows():
        marker = " ← BEST" if idx == comparison_df.index[0] else ""
        print(f"{row['Algorithm']:<25} {row['Silhouette']:<15.4f} {row['Balance']:<12.3f} "
              f"{row['Calinski-H']:<15.2f} {row['Davies-B']:<12.4f}{marker}")
    
    best_algo = comparison_df.iloc[0]['Algorithm']
    best_sil = comparison_df.iloc[0]['Silhouette']
    
    print(f"\n✓ Best Algorithm: {best_algo}")
    print(f"✓ Silhouette Score: {best_sil:.4f}")

## 9. Summary & Next Steps

In [ ]:
if X_processed is not None:
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"\n✓ Successfully clustered {len(X_processed)} stocks")
    print(f"✓ Achieved silhouette score: {optimal_score:.4f}")
    print(f"✓ Number of clusters: {optimal_k}")
    print(f"✓ PCA components used: {pca_model.n_components_}")
    print(f"\n📁 Generated Files:")
    print(f"   1. 01_raw_features.xlsx")
    print(f"   2. 02_preprocessed_features.csv")
    print(f"   3. 03_optimized_clusters_k{optimal_k}_score{optimal_score:.3f}.xlsx")
    print(f"   4. 04_visualization_k{optimal_k}_score{optimal_score:.3f}.png")
    
    print(f"\n🎯 Next Steps for Portfolio Optimization:")
    print(f"   1. Select representative stocks from each cluster")
    print(f"   2. Apply diversification constraints")
    print(f"   3. Run mean-variance optimization")
    print(f"   4. Backtest the portfolio")
    
    print("\n" + "="*80)
else:
    print("\n⚠ Clustering failed. Please check your data.")